<a href="https://colab.research.google.com/github/micah-shull/pipelines/blob/main/pipelines_16_ensemble_048_optimal_thresholds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classifiers with Custom Threshold or Regularization

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import warnings
from loan_data_utils import load_and_preprocess_data, plot_class_distribution, plot_mean_class_metrics, get_top_performers, evaluate_model

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Example usage with custom thresholds
stacking_threshold = 0.3  # Set your custom threshold for stacking classifier
voting_threshold = 0.3    # Set your custom threshold for voting classifier

# Define your URL, categorical columns, and target
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'

# Load and preprocess data
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Define models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import HistGradientBoostingClassifier

def define_models(best_params):
    models = {
        "Logistic Regression (ADASYN)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', ADASYN()),
            ('classifier', LogisticRegression(random_state=42, **best_params['Class 1 Recall']['Logistic Regression (ADASYN)']['best_params']))
        ]),
        "Logistic Regression (SMOTE)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', SMOTE()),
            ('classifier', LogisticRegression(random_state=42, **best_params['Class 1 Recall']['Logistic Regression (SMOTE)']['best_params']))
        ]),
        "LGBM (SMOTE)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', SMOTE()),
            ('classifier', LGBMClassifier(random_state=42, **best_params['Class 1 Recall']['LGBM (SMOTE)']['best_params'], force_row_wise=True))
        ]),
        "Logistic Regression (baseline)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(random_state=42, **best_params['Class 1 Precision']['Logistic Regression (baseline)']['best_params']))
        ]),
        "LGBM (baseline)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LGBMClassifier(random_state=42, **best_params['Class 1 Precision']['LGBM (baseline)']['best_params'], force_row_wise=True))
        ]),
        "Random Forest (class_weight_balanced)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', **best_params['Class 1 Precision']['Random Forest (class_weight_balanced)']['best_params']))
        ]),
        "Logistic Regression (baseline for Class 0 Recall)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(random_state=42, **best_params['Class 0 Recall']['Logistic Regression (baseline)']['best_params']))
        ]),
        "LGBM (baseline for Class 0 Recall)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LGBMClassifier(random_state=42, **best_params['Class 0 Recall']['LGBM (baseline)']['best_params'], force_row_wise=True))
        ]),
        "Random Forest (class_weight_balanced for Class 0 Recall)": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', **best_params['Class 0 Recall']['Random Forest (class_weight_balanced)']['best_params']))
        ]),
        "LGBM (RandomUnderSampler)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', RandomUnderSampler()),
            ('classifier', LGBMClassifier(random_state=42, **best_params['Class 0 Precision']['LGBM (RandomUnderSampler)']['best_params'], force_row_wise=True))
        ]),
        "HistGradientBoosting (RandomUnderSampler)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', RandomUnderSampler()),
            ('classifier', HistGradientBoostingClassifier(random_state=42, **best_params['Class 0 Precision']['HistGradientBoosting (RandomUnderSampler)']['best_params']))
        ]),
        "Random Forest (RandomUnderSampler)": ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', RandomUnderSampler()),
            ('classifier', RandomForestClassifier(random_state=42, **best_params['Class 0 Precision']['Random Forest (RandomUnderSampler)']['best_params']))
        ])
    }
    return models

# Train and evaluate classifiers
def save_results(results, filename):
    # Strip 'optimal_model_params_thresholds_' and '.json' from the filename
    method_name = filename.replace('optimal_model_params_thresholds_', '').replace('.json', '')
    results_filename = f'voting_stacking_results_{method_name}.json'

    # Save the results to a JSON file
    with open(results_filename, 'w') as file:
        json.dump(results, file, indent=4)

# Function to predict with a custom threshold
def predict_with_threshold(model, X, threshold=0.5):
    probas = model.predict_proba(X)[:, 1]
    return (probas >= threshold).astype(int)

def train_and_evaluate_voting_stacking(models, X_train, y_train, X_test, y_test, filename, stacking_threshold=0.5, voting_threshold=0.5):
    voting_clf = VotingClassifier(estimators=list(models.items()), voting='soft')
    stacking_clf = StackingClassifier(estimators=list(models.items()), final_estimator=LogisticRegression(C=0.01, penalty='l2', solver='liblinear', random_state=42))

    # Train the classifiers
    voting_clf.fit(X_train, y_train)
    stacking_clf.fit(X_train, y_train)

    # Predict with custom thresholds
    y_pred_voting = predict_with_threshold(voting_clf, X_test, voting_threshold)
    y_pred_stacking = predict_with_threshold(stacking_clf, X_test, stacking_threshold)

    # Evaluate performance
    print("Voting Classifier Performance with Custom Threshold:")
    print(classification_report(y_test, y_pred_voting))

    print("Stacking Classifier Performance with Custom Threshold:")
    print(classification_report(y_test, y_pred_stacking))

    # Prepare results
    results = {
        'voting': classification_report(y_test, y_pred_voting, output_dict=True),
        'stacking': classification_report(y_test, y_pred_stacking, output_dict=True)
    }

    # Save the results
    save_results(results, filename)

## Train & Evaluate Classifiers

In [5]:
def train_classifiers_with_optimal_params(files, X_train, y_train, X_test, y_test):
    for filename in files:
        # Load optimal parameters and thresholds
        with open(filename, 'r') as file:
            best_params = json.load(file)

        # Define models
        models = define_models(best_params)

        # Train and evaluate classifiers
        train_and_evaluate_voting_stacking(models, X_train, y_train, X_test, y_test, filename, stacking_threshold, voting_threshold)

        # Ensure the results file is saved correctly
        results_filename = f'voting_stacking_results_{filename.replace("optimal_model_params_thresholds_", "").replace(".json", "")}.json'
        print(f'Results saved to {results_filename}')

# List of files
files = [
    'optimal_model_params_thresholds_custom_loss.json',
    'optimal_model_params_thresholds_f1.json',
    'optimal_model_params_thresholds_grid_search.json',
    'optimal_model_params_thresholds_recall.json',
    'optimal_model_params_thresholds_roc.json'
]

# Call the function with the list of files and data
train_classifiers_with_optimal_params(files, X_train, y_train, X_test, y_test)

#-----------------

# # Load optimal parameters and thresholds
# filename = 'optimal_model_params_thresholds_f1.json'
# with open(filename, 'r') as file:
#     best_params = json.load(file)

# # define model params
# models = define_models(best_params)

# train_and_evaluate_voting_stacking(models, X_train, y_train, X_test, y_test, filename, stacking_threshold, voting_threshold)

# # Ensure the results file is saved correctly
# results_filename = f'voting_stacking_results_{filename.replace("optimal_model_params_thresholds_", "").replace(".json", "")}.json'
# print(f'Results saved to {results_filename}')


[LightGBM] [Info] Number of positive: 18691, number of negative: 18691
[LightGBM] [Info] Total Bins 6553
[LightGBM] [Info] Number of data points in the train set: 37382, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221208 -> initscore=-1.258639
[LightGBM] [Info] Start training from score -1.258639
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221208 -> initscore=-1.258639
[LightGBM] [Info] Start training from score -1.258639
[LightGBM] [Info] Number of positive: 5309, number of negativ